## Experiments: Sensitive Analysis Hyperparameter RSAST:


It is runned RSAST in a set of UCR datasets with a predefined number of runs ("runs"). Then, it is selected a range ("range_total") between 1, 10, 30 ,50 and 100 for the selected dataset.

In [1]:
#configure directory to import sast libraries
import sys 
import os 
#add sast library path
file_path = os.path.expanduser("~/random_sast/sast")
sys.path.append(file_path)
#add cd_diagram library path
file_path = os.path.expanduser("~/random_sast/cd_diagram")
sys.path.append(file_path)

#sys.path

In [2]:
from sast import *
from sktime.datasets import load_UCR_UEA_dataset, tsc_dataset_names
from sktime.classification.kernel_based import RocketClassifier
import time
import pandas as pd
import os

c:\Users\Surface pro\anaconda3\lib\site-packages\mass_ts\_mass_ts.py:17: UserWarning: GPU support will not work. You must pip install mass-ts[gpu].
  warnings.warn(


### Reading Datasets

In [3]:
# Set directory where the csv files are located
directory = os.getcwd()+'/ResultsByClassifier'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        # Append the dataframe to the list
        df['filename']=filename
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_other_methods = pd.concat(dfs, ignore_index=True)
df_other_methods=df_other_methods[['filename','folds:','0']]
df_other_methods['method']=df_other_methods['filename'].str.split('_').str[0]
df_other_methods
df_other_methods=df_other_methods.rename(columns={'folds:':'dataset','0':'score'})
df_other_methods=df_other_methods[['dataset','score','method']]


In [4]:
df_other_methods['method'].unique()

array(['BOSS', 'Catch22', 'cBOSS', 'HIVE-COTEv1', 'InceptionTime',
       'ProximityForest', 'ResNet', 'RISE', 'ROCKET', 'S-BOSS', 'SAST',
       'STC', 'TS-CHIEF', 'TSF', 'WEASEL'], dtype=object)

In [5]:
# Set directory where the csv files are located
directory = os.getcwd()+'/ResultsRsast'
#directory = os.getcwd()+'/results_accuracy_per_ds'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.startswith("df_overall_tunning") and filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        df['rpoint']=df['rpoint'].astype(str)
        df['nb_per_class']=df['nb_per_class'].astype(str)

        
        # Append the dataframe to the list
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_result = pd.concat(dfs, ignore_index=True)
df_result.head(3)


,Unnamed: 0,accuracy,time,cweights_time,fsubsequence_time,tdataset_time,tclassifier_time,dataset_name,classifier_name,rpoint,nb_per_class,method,len_method
0,0,0.9,22.449903,0.118879,18.913046,3.344095,0.073366,BirdChicken,ACF&PACF: n_random_points=1 nb_inst_per_class=1,1,1,Rsast,ACF&PACF
1,1,0.9,0.423205,0.104959,0.011462,0.302871,0.003022,BirdChicken,ACF: n_random_points=1 nb_inst_per_class=1,1,1,Rsast,ACF
2,2,0.7,18.675146,0.104006,18.243167,0.323289,0.003877,BirdChicken,PACF: n_random_points=1 nb_inst_per_class=1,1,1,Rsast,PACF


In [6]:
#df_result=pd.read_csv("df_overall_tunning_SmoothSubspace3.csv")
df_result['dataset_name'].unique()


array(['BirdChicken', 'Chinatown', 'Coffee',
       'DistalPhalanxOutlineAgeGroup', 'DistalPhalanxOutlineCorrect',
       'GunPoint', 'ItalyPowerDemand', 'ShapeletSim', 'SmoothSubspace',
       'SonyAIBORobotSurface1', 'SonyAIBORobotSurface2',
       'SyntheticControl'], dtype=object)

In [7]:
df_result.columns.unique()

Index(['Unnamed: 0', 'accuracy', 'time', 'cweights_time', 'fsubsequence_time',
       'tdataset_time', 'tclassifier_time', 'dataset_name', 'classifier_name',
       'rpoint', 'nb_per_class', 'method', 'len_method'],
      dtype='object')

### Hyperparameter Tunning RSAST 

#### Accuracy: Subsequence Lenght Method 

In [8]:
# create a pivot table with the mean of score by dataset
len_method_vs_ds=pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['dataset_name'], aggfunc='mean')
len_method_vs_ds=np.transpose(len_method_vs_ds)#[['ACF','PACF']]
len_method_vs_ds=len_method_vs_ds.reset_index()
df_rocket=df_other_methods[df_other_methods["method"]=="ROCKET"]
merged_df = len_method_vs_ds.merge(df_rocket,left_on='dataset_name', right_on='dataset',  how='left')
merged_df=merged_df.drop('method',axis=1)
merged_df

,dataset_name,ACF,ACF&PACF,Max ACF,Max PACF,None,PACF,dataset,score
0,BirdChicken,0.776667,0.830000,0.713333,0.755333,0.708667,0.837333,BirdChicken,0.900000
1,Chinatown,0.952624,0.964577,0.944485,0.962707,0.950777,0.965719,Chinatown,0.982507
2,Coffee,0.939286,0.998214,0.953571,0.937500,0.991071,0.995536,Coffee,1.000000
3,DistalPhalanxOutlineAgeGroup,0.613813,0.641823,0.559808,0.551367,0.543789,0.638082,DistalPhalanxOutlineAgeGroup,0.748201
4,DistalPhalanxOutlineCorrect,0.450097,0.433671,0.476039,0.452029,0.472850,0.454976,DistalPhalanxOutlineCorrect,0.771739
5,GunPoint,0.790722,0.973167,0.798056,0.955222,0.929111,0.967444,GunPoint,1.000000
6,ItalyPowerDemand,0.908698,0.917080,0.902098,0.899992,0.908107,0.913443,ItalyPowerDemand,0.968902
7,ShapeletSim,0.944296,0.891037,0.895704,0.734593,0.631259,0.881037,ShapeletSim,1.000000
8,SmoothSubspace,0.823500,0.868444,0.823556,0.857500,0.829389,0.869722,SmoothSubspace,0.980000
9,SonyAIBORobotSurface1,0.742696,0.789063,0.743738,0.722374,0.739567,0.789040,SonyAIBORobotSurface1,0.921797


In [9]:
# create a pivot table with the mean of score by number of random points
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['rpoint'], aggfunc='mean')

rpoint,(lenthg ts)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.867221,0.689606,0.759221,0.769812,0.754094,0.766699
ACF&PACF,0.935906,0.769768,0.793693,0.802230,0.795402,0.802671
Max ACF,0.866565,0.679753,0.743472,0.751705,0.727673,0.758815
Max PACF,0.921676,0.698584,0.765281,0.767215,0.748938,0.765555
None,0.903498,0.697943,0.744425,0.744596,0.734490,0.742230
PACF,0.934887,0.773026,0.787770,0.806975,0.804790,0.800164


In [10]:
# create a pivot table with the mean of score by number of instances per class
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.862548,0.687374,0.745829,0.774323,0.763774,0.772152
ACF&PACF,0.937923,0.760589,0.782889,0.808394,0.804014,0.806142
Max ACF,0.864530,0.646907,0.744624,0.758136,0.749201,0.764300
Max PACF,0.924309,0.682373,0.767379,0.768072,0.757129,0.768356
None,0.909010,0.664852,0.742451,0.757422,0.740565,0.753654
PACF,0.936756,0.764501,0.784894,0.804212,0.811534,0.805977


#### Time: Subsequence Lenght Method 

In [11]:
# create a pivot table with the mean of the time by dataset
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['dataset_name'], aggfunc='mean')

dataset_name,BirdChicken,Chinatown,Coffee,DistalPhalanxOutlineAgeGroup,DistalPhalanxOutlineCorrect,GunPoint,ItalyPowerDemand,ShapeletSim,SmoothSubspace,SonyAIBORobotSurface1,SonyAIBORobotSurface2,SyntheticControl
len_method,,,,,,,,,,,,
ACF,81.630738,0.120657,43.827708,132.824072,177.888586,42.810175,0.674857,7.645123,1.599955,0.419210,0.815245,128.598058
ACF&PACF,297.280662,0.368780,148.320089,419.191777,519.712858,95.588660,1.649303,222.211859,2.648634,2.247746,3.008572,402.314136
Max ACF,2.745482,0.122675,3.134972,49.805655,59.238734,4.674037,0.681721,2.640422,1.595359,0.402863,0.627533,45.391469
Max PACF,107.885490,0.134552,6.470813,29.798286,35.902560,4.200933,0.747006,124.217464,1.849311,0.405555,0.495278,29.918500
None,1.853694,0.109866,1.756151,28.495936,34.720866,2.811864,0.665972,1.836391,1.571106,0.300002,0.397229,28.232846
PACF,208.928276,0.290148,105.678870,293.491555,345.791509,52.941237,1.651981,215.296044,2.606331,1.952652,2.348842,260.544206


In [12]:
# create a pivot table with the mean of the time by number of instances per class
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
len_method,,,,,,
ACF,10.478092,2.393111,23.687626,99.101473,49.338871,68.890502
ACF&PACF,26.537346,9.860271,94.075738,319.731423,173.906681,229.309919
Max ACF,1.425696,0.578089,4.924781,30.495897,13.094834,19.929481
Max PACF,1.470584,2.604420,25.396368,40.373686,34.213613,34.639751
None,0.936159,0.406352,3.102381,18.182226,8.024046,11.925666
PACF,16.443605,7.130951,70.660643,221.672740,121.357824,161.044083


In [13]:
# create a pivot table with the mean of the time by number of random points
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['rpoint'], aggfunc='mean')

rpoint,(lenthg ts)//2,1,10,100,30,50
len_method,,,,,,
ACF,17.353777,1.408392,12.691002,123.744379,37.460210,62.193031
ACF&PACF,44.615108,22.204273,55.632310,389.614995,131.734754,212.146936
Max ACF,1.903179,0.478671,3.718879,35.747113,10.732377,17.935304
Max PACF,1.755119,17.963555,19.463035,41.796966,26.612665,31.146855
None,1.166718,0.305105,2.263935,21.727218,6.533754,10.612330
PACF,27.683812,20.634828,42.571903,267.315834,96.307789,145.366890


#### Variance: Subsequence Lenght Method 

In [14]:
# create a pivot table with the variance of the accuracy by number of instances per class
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['nb_per_class'], aggfunc='var')

nb_per_class,(max instances per class)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.007970,0.041785,0.048042,0.041803,0.043012,0.042357
ACF&PACF,0.001913,0.044637,0.050884,0.044363,0.045568,0.046923
Max ACF,0.008381,0.049628,0.047281,0.047458,0.050565,0.044454
Max PACF,0.002455,0.058892,0.048996,0.052683,0.056313,0.053442
None,0.004761,0.053863,0.053352,0.050767,0.056502,0.050467
PACF,0.001814,0.040651,0.050335,0.046624,0.039819,0.045596


In [15]:
# create a pivot table with the variance of the accuracy by number of random points
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['rpoint'], aggfunc='var')

rpoint,(lenthg ts)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.008263,0.050129,0.043716,0.038893,0.043954,0.039921
ACF&PACF,0.002802,0.051221,0.046376,0.044188,0.046525,0.044766
Max ACF,0.008748,0.050992,0.047467,0.043817,0.058007,0.043782
Max PACF,0.003517,0.063361,0.049815,0.049245,0.060247,0.049718
None,0.006059,0.061903,0.051679,0.048657,0.056064,0.051440
PACF,0.002525,0.045075,0.050806,0.040169,0.042459,0.045075


#### Another Results

In [16]:
df_result[['dataset_name','len_method','classifier_name']].describe()

,dataset_name,len_method,classifier_name
count,6540,6540,6540
unique,12,6,216
top,Chinatown,ACF&PACF,None: n_random_points= (lenthg ts)//2 nb_inst_...
freq,720,1090,80


In [17]:
pd.pivot_table(df_result, values='time', index=['dataset_name'], aggfunc='sum')/(60*60)

,time
dataset_name,
BirdChicken,14.590090
Chinatown,0.038223
Coffee,3.435429
DistalPhalanxOutlineAgeGroup,19.866818
DistalPhalanxOutlineCorrect,24.442815
GunPoint,6.767564
ItalyPowerDemand,0.202361
ShapeletSim,11.955152
SmoothSubspace,0.395690


In [18]:
df_result['time'].sum()/(60*60)

100.71571616550287

#### Generate CD Diagram Tunning Hyperparameter

In [19]:
#create dataset to generate cd diagram
df_perf=pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')
df_perf=df_perf.reset_index()
#df_perf.classifier_name.unique()
df_perf[df_perf.classifier_name.str.contains(r'^ACF&PACF:')].head(5)

,classifier_name,dataset_name,accuracy
0,ACF&PACF: n_random_points= (lenthg ts)//2 nb_i...,Chinatown,0.973178
1,ACF&PACF: n_random_points= (lenthg ts)//2 nb_i...,Coffee,1.000000
2,ACF&PACF: n_random_points= (lenthg ts)//2 nb_i...,GunPoint,0.979556
3,ACF&PACF: n_random_points= (lenthg ts)//2 nb_i...,ItalyPowerDemand,0.945578
4,ACF&PACF: n_random_points= (lenthg ts)//2 nb_i...,SmoothSubspace,0.888444


In [20]:
from cd_function import *
#generate hyperparameter tuning cd diagrams
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^ACF&PACF:')], labels=True, title="ACF&PACF comparison", name_file='images_cd_diagram/cd-diagram_ACF&PACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^PACF:')], labels=True, title="PACF comparison", name_file='images_cd_diagram/cd-diagram_PACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^ACF:')], labels=True, title="ACF comparison", name_file='images_cd_diagram/cd-diagram_ACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^None:')], labels=True, title="None comparison", name_file='images_cd_diagram/cd-diagram_None.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^Max ACF:')], labels=True, title="Max ACF comparison", name_file='images_cd_diagram/cd-diagram_MaxACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^Max PACF:')], labels=True, title="Max PACF comparison", name_file='images_cd_diagram/cd-diagram_MaxPACF.png')
draw_cd_diagram(df_perf, labels=True, title="overall Hyperparameter Tunning", name_file='images_cd_diagram/cd-diagram_overall_tunning.png')



c:\Users\Surface pro\anaconda3\lib\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
c:\Users\Surface pro\anaconda3\lib\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
c:\Users\Surface pro\anaconda3\lib\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
c:\Users\Surface pro\anaconda3\lib\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-val

In [21]:
_, average_ranks, _ = wilcoxon_holm(df_perf=df_perf)
min_rank= min(average_ranks)
average_ranks=pd.DataFrame(average_ranks)
average_ranks

c:\Users\Surface pro\anaconda3\lib\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


,0
Max PACF: n_random_points=30 nb_inst_per_class=1,127.916667
Max ACF: n_random_points=30 nb_inst_per_class=1,122.083333
Max ACF: n_random_points=1 nb_inst_per_class=1,121.583333
Max PACF: n_random_points=100 nb_inst_per_class=1,118.000000
Max ACF: n_random_points=10 nb_inst_per_class=1,117.541667
...,...
ACF&PACF: n_random_points=100 nb_inst_per_class=50,43.333333
ACF&PACF: n_random_points=100 nb_inst_per_class=30,42.958333
ACF&PACF: n_random_points=10 nb_inst_per_class=100,41.708333
ACF&PACF: n_random_points=50 nb_inst_per_class=100,39.416667


In [22]:
#generate example cd diagrams
draw_cd_diagram(pd.read_csv('example_cd_diagram.csv'), labels=True, title="Example CD Diagram", name_file='cd-diagram_example.png')

c:\Users\Surface pro\anaconda3\lib\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


#### Focus on most accurate lenght method

##### Overall Accuracy

In [23]:
# highest accurate hyperparameters
average_ranks[average_ranks[0]==min_rank][0].index

Index(['PACF: n_random_points=100 nb_inst_per_class=30'], dtype='object')

In [24]:
# filter by method with ACF&PACF
# create a pivot table with the mean of score by hyperparameter
df_result_acc=df_result[df_result["len_method"]=="ACF&PACF"]
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.940985,0.870623,0.932170,0.950176,0.948727,0.952445
1,0.922227,0.714988,0.696115,0.789771,0.790266,0.785953
10,0.937866,0.727182,0.790894,0.796666,0.795204,0.790671
100,0.940391,0.777466,0.786825,0.792186,0.795236,0.794422
30,0.935181,0.746645,0.782546,0.795352,0.793410,0.793275
50,0.938645,0.784883,0.787817,0.801271,0.777855,0.797541


In [25]:
# create a pivot table with the variatioon of score by hyperparameter
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='var')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.001667,0.008885,0.003010,0.001296,0.001327,0.001025
1,0.002817,0.062568,0.077627,0.040615,0.037238,0.046755
10,0.002029,0.059836,0.040519,0.047997,0.045891,0.050160
100,0.001802,0.035086,0.047333,0.052971,0.049867,0.051297
30,0.002857,0.046467,0.055026,0.048539,0.045650,0.051740
50,0.001660,0.031979,0.044719,0.047246,0.064878,0.051205


##### Overall time complexity

In [26]:
# create a pivot table with the mean of time spent in minutes
pd.pivot_table(df_result_acc, values=['time','cweights_time','fsubsequence_time','tdataset_time','tclassifier_time'], index=['dataset_name'], aggfunc='mean')/60

,cweights_time,fsubsequence_time,tclassifier_time,tdataset_time,time
dataset_name,,,,,
BirdChicken,0.001775,1.845793,0.000515,3.106501,4.954678
Chinatown,0.000153,0.001468,0.000136,0.004364,0.006146
Coffee,0.001170,0.102598,0.000719,2.367384,2.472001
DistalPhalanxOutlineAgeGroup,0.000686,0.038889,0.010279,6.936574,6.986530
DistalPhalanxOutlineCorrect,0.000844,0.028763,0.012983,8.619190,8.661881
GunPoint,0.000702,0.035006,0.001057,1.556273,1.593144
ItalyPowerDemand,0.000133,0.002412,0.000269,0.024644,0.027488
ShapeletSim,0.002136,2.069743,0.000366,1.631211,3.703531
SmoothSubspace,0.000103,0.002886,0.000560,0.040557,0.044144


In [27]:
# create a pivot table with the mean of time spent in minutes by random points and instances per class
pd.pivot_table(df_result_acc, values='time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')/60

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.610886,0.060577,0.651150,1.231233,1.223211,1.215250
1,0.026830,0.056340,0.424776,0.516456,0.507142,0.507167
10,0.080599,0.078892,0.720324,1.747770,1.123094,1.364348
100,0.693178,0.375743,3.697348,14.121683,6.960061,10.042684
30,0.216818,0.140927,1.359760,4.634129,2.447615,3.326647
50,0.351035,0.218616,2.068863,7.552542,4.242656,5.094940


##### Calculate weights time complexity

In [28]:
pd.pivot_table(df_result_acc, values='cweights_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.019112,0.019674,0.018949,0.019782,0.017922,0.019191
1,0.019525,0.039505,0.034749,0.043230,0.037325,0.038426
10,0.017632,0.041367,0.036154,0.046714,0.043347,0.042696
100,0.017201,0.052451,0.043583,0.046503,0.049913,0.050062
30,0.019130,0.044675,0.042425,0.049988,0.046137,0.050697
50,0.020988,0.046173,0.046191,0.052046,0.046225,0.050087


##### Finding subsequences time complexity

In [29]:
pd.pivot_table(df_result_acc, values='fsubsequence_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.520645,0.068894,0.606605,0.912468,1.019892,0.905657
1,0.651737,2.466725,23.405273,22.474001,26.292221,24.590836
10,0.614112,2.626947,23.550564,22.773997,26.564345,25.116368
100,0.503716,2.853488,27.140589,27.841491,25.001743,38.272257
30,0.514287,2.396395,22.864531,28.764379,27.377542,29.920977
50,0.462872,2.856334,28.456786,26.645855,30.253524,26.800147


##### Transform Dataset time complexity

In [30]:
pd.pivot_table(df_result_acc, values='tdataset_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,36.086862,3.536457,38.416659,72.895030,72.312915,71.947842
1,0.926862,0.851994,1.973064,8.379904,4.031182,5.722793
10,4.190709,2.011591,19.536340,81.891388,40.658658,56.582710
100,41.039841,19.530270,194.459079,817.589168,391.597881,562.884826
30,12.457496,5.938078,58.572563,247.579255,119.253728,169.298435
50,20.557796,10.107716,95.477171,408.686579,198.648432,278.487083


##### Classifier time complexity

In [31]:
pd.pivot_table(df_result_acc, values='tclassifier_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.021252,0.008406,0.023765,0.039768,0.036790,0.037289
1,0.010938,0.019162,0.072361,0.088792,0.066804,0.076676
10,0.011919,0.052410,0.095189,0.151160,0.117306,0.116650
100,0.027472,0.104079,0.191388,1.807877,0.944107,1.341054
30,0.016138,0.074968,0.103886,1.646955,0.174793,0.323106
50,0.018168,0.103675,0.147375,17.759222,25.605202,0.350316
